# A real world example: New Database extension

#### Why build this?
I built the New Database extension because we need empty DBs to test against all the time. We can use the `sqlCreateDatabase` snippet, but when done a _lot_ having to open a query window and type / execute a snippet felt too slow. I want a new database now!
> This extension was also a great way to show the power of extensions since it connects, issues a SQL query, and kicks off a background task.

### Install the extension
* <a href="command:workbench.view.extensions">View Extensions</a> and Search **New Database**
* Click install, download VSIX and use **Install from VSIX** command to install

### Create a new database

|Step| Contribution Point | What's added | 
|----|----|----|
|Right-click on the Databases folder for a connection| Menus |  **New Database** menu item, when node label is Databases| 
|Choose **New Database** | Command |  **New Database** command | 
|Enter database name| Quick Pick menu | Request simple input | 
|See task progress| Tasks window | Create Database started/succeeded/failed | 
|Notify with more information / next steps| Show Message | | 

<br/>



## The Data Management Protocol (DMP)

To create a database something needs to connect to a DB and run a SQL command. You could write your own backend in Typescript, .Net Core, Python etc. but why do this when we already have connect, query and many more actions built-in?

The Data Management Protocol (DMP) defines the APIs needed to connect, query, and more. 
* SQL Server and PostgreSQL connections work using the DMP and run in extensions
* All the core operations in Azure Data Studio use the DMP
* Using this avoids issues where your driver doesn't support an authentication type (e.g. Azure Active Directory), so it's a great idea to use it!

| Action | DMP Provider | Notes |
|----|----|----|
| Connect to a SQL DB | **ConnectionProvider**| |
| Run a query | **QueryProvider** | |
| Object Explorer / Schema Browsing |  **ObjectExplorerProvider**  | Primary provider of schema / metadata about a connection in the **Connections** view. Should provide all the basic schema information, management folders and more |
| Add nodes under an Object Explorer session  | **ObjectExplorerNodeProvider** | Augment SQL Server or other connection type with additional subtrees. For example: Data Services & HDFS folders under SQL Server connections use this since they're using REST instead of TDS to query metadata. Great for adding Monitoring, customized metadata, or other features. |
| Script objects | **ScriptingProvider** | |




## Walkthrough: Writing the New Database extension


In [1]:
!code ./3-newdatabase/


### 1. Define new command

#### package.json
* Add a command `newdatabase.createdb`
* Show in on right-click if we're on the Databases node

```json

"activationEvents": [
    "onCommand:newdatabase.createdb"
],
"contributes": {
    "commands": [
        {
            "command": "newdatabase.createdb",
            "title": "New Database"
        }
    ],
    "menus": {
        "objectExplorer/item/context": [
            {
                "command": "newdatabase.createdb",
                "when": "connectionProvider == MSSQL && nodeType && nodeType == Folder && nodeLabel=~/^Databases$/",
                "group": "1data1"
            }
        ]
    },
    "keybindings": [
        {
            "command": "newdatabase.createdb",
            "key": "Ctrl+Shift+d b"
        }
    ]
},
```



### 2. Define the contribution in extension.ts

```ts
export function activate(context: vscode.ExtensionContext) {
    context.subscriptions.push(vscode.commands.registerCommand('newdatabase.createdb', async (context: sqlops.ObjectExplorerContext) => {
        createDatabase(context);
    }));
}
```



### 3. Basic task flows

The basic flow of any task is to verify inputs, prompt a user if anything extra is needed, then kick off a task. This is really easy using the Azure Data Studio APIs.

```ts
async function createDatabase(context: sqlops.ObjectExplorerContext): Promise<void> {
    let connection = await makeSureWeAreConnected(context);
    if (!connection) {
        vscode.window.showInformationMessage('Cannot create database as no active connection could be found');
        return;
    }

    // Prompt the user for a new database name
    let dbName = await vscode.window.showInputBox({
        prompt: 'Name of database to create on server ' + connection.options['server']
    });
    if (!dbName) {
        return;
    }

    // Run the create database as a task
    sqlops.tasks.startBackgroundOperation({
        connection: connection as sqlops.connection.Connection,
        displayName: 'Creating Database ' + dbName,
        description: '',
        isCancelable: false,
        operation: (op) => doCreateDatabase(op, dbName, connection)
    });
}
```

### Using the DMP to connect

Connection is a 2-step process in the DMP. To simplify things until we get new APIs ready, I created a helper package to create a new connection for use with queries.

**package.json**
```json
"dependencies": {
    "azuredatastudio-dmpwrapper": "^0.5.0"
}
```

**extension.ts**

Import the helper:
```ts
import { ConnectionContext } from 'azuredatastudio-dmpwrapper';
```

Then create it after getting the right provider
```ts
let connectionProvider = sqlops.dataprotocol.getProvider<sqlops.ConnectionProvider>(
    'MSSQL', sqlops.DataProviderType.ConnectionProvider);
let connectionContext = new ConnectionContext(connectionProvider);
```

Run connection before any queries 
```ts
let isConnected = await connectionContext.tryConnect(connection);
```

And dispose the connection when you're done - be sure to put this in a `finally` block!
```ts
} finally {
    connectionContext.dispose();
}
```

### Using the DMP to run a simple query

The `runQueryAndReturn` method is the simplest and best one right now. We will add a more robust API in the future the one drawback is you need to return a resultset.

Note you should also try to sanitize inputs, per the example below!

```ts
dbName = dbName.replace(/]/g , "]]");
let query = 'BEGIN TRY
    CREATE DATABASE [' + dbName + `]
    SELECT 1 AS NoError
END TRY
BEGIN CATCH
    SELECT ERROR_MESSAGE() AS ErrorMessage;
END CATCH
`;

let result = await connectionContext.runQueryAndReturn(query);
```


### Completing the task

For task to complete, set the status to Succeeded or Failed:
```ts
try {
    ...
    let successMsg = 'Database ' + dbName + ' created. Refresh the Databases node to see it';
    operation.updateStatus(sqlops.TaskStatus.Succeeded, successMsg);
} catch (error) {
    let errorString = error instanceof Error ? error.message : error;
    operation.updateStatus(sqlops.TaskStatus.Failed, 'Error adding database: ' + errorString);
}

```

### Summary

By now, you've seen how to use advanced APIs in Azure Data Studio. There are a lot more, including wizards, dialogs and other UI. Check out the [Azure Data Studio wiki on Extension Authoring](https://github.com/Microsoft/azuredatastudio/wiki/Extension-Authoring) and our [Samples projects](https://github.com/microsoft/azuredatastudio/tree/master/samples) to learn more.